<a href="https://colab.research.google.com/github/nicolasvazquez95/Aprendiendo_DeepLearning/blob/main/10_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data - Tokenization and Embedding

In [1]:
# Imports
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# Helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
import helper_functions as helper

--2022-05-16 16:34:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-16 16:34:50 (60.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Get the text dataset
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
helper.unzip_data('nlp_getting_started.zip')

--2022-05-16 16:34:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.167.128, 64.233.166.128, 74.125.71.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.167.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-05-16 16:34:51 (104 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Shuffle
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X = train_df_shuffled['text'].copy()
y = train_df_shuffled['target'].copy()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [6]:
## Tokenization
from tensorflow.keras.layers import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [7]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(X_train)

In [8]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[252,   3, 244,   4,  13, 727,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [9]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['noahanyname', 'noah', 'no2', 'nnw', 'nno']


In [10]:
## Embedding Layer
embedding = keras.layers.Embedding(input_dim=max_vocab_length,
                                   output_dim=128,
                                   input_length=max_length)

In [11]:
import random
random_sentence = random.choice(X_train)
print(f'Original text:\n {random_sentence}\
\n\nEmbedded version:\n')
#Embedded version
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 I know it's a question of interpretation but this is a sign of the apocalypse.  I called it https://t.co/my8q1uWIjn

Embedded version:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01142566,  0.00125264, -0.02954164, ...,  0.0331663 ,
         -0.00307534, -0.04909436],
        [ 0.00311036,  0.00983224,  0.04595358, ..., -0.01868846,
          0.00816754, -0.02888161],
        [-0.01222702, -0.02359438, -0.02720351, ...,  0.04552272,
          0.00627636, -0.00946717],
        ...,
        [ 0.02290528,  0.02146382, -0.04418736, ..., -0.0267189 ,
          0.02797374, -0.03323121],
        [ 0.00167676,  0.01227218, -0.00221572, ...,  0.01482412,
          0.0159439 ,  0.0439803 ],
        [ 0.03809874,  0.0365029 , -0.02225101, ..., -0.04683873,
          0.03840974, -0.01186135]]], dtype=float32)>

# Model 0 : Naive Bayes (Scikit)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([('tfidf',TfidfVectorizer()),
                    ('clf',MultinomialNB())])
model_0.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [13]:
model_0.score(X_test,y_test)

0.8003502626970228

In [14]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [15]:
# Get baseline results
y_pred_0 = model_0.predict(X_test)

baseline_results = calculate_results(y_true=y_test,
                                     y_pred=y_pred_0)
baseline_results

{'accuracy': 80.03502626970229,
 'f1': 0.7937090801534213,
 'precision': 0.8170270320769228,
 'recall': 0.8003502626970228}

# Model 1 : Simple Dense model

In [16]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = 'model_logs'

# Build model Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_1 = keras.Model(inputs,outputs)

model_1.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [17]:
model_1.fit(X_train,y_train,
            epochs=5,
            validation_data=(X_test,y_test),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'model_1_dense')]
            )

Saving TensorBoard log files to: model_logs/model_1_dense/20220516-163459
Epoch 1/5
203/203 [==============================] - 9s 14ms/step - loss: 0.6184 - accuracy: 0.6885 - val_loss: 0.5442 - val_accuracy: 0.7461
Epoch 2/5
203/203 [==============================] - 2s 11ms/step - loss: 0.4479 - accuracy: 0.8163 - val_loss: 0.4774 - val_accuracy: 0.7846
Epoch 3/5
203/203 [==============================] - 2s 11ms/step - loss: 0.3493 - accuracy: 0.8609 - val_loss: 0.4658 - val_accuracy: 0.7907
Epoch 4/5
203/203 [==============================] - 3s 12ms/step - loss: 0.2859 - accuracy: 0.8881 - val_loss: 0.4674 - val_accuracy: 0.7890
Epoch 5/5
203/203 [==============================] - 2s 12ms/step - loss: 0.2386 - accuracy: 0.9104 - val_loss: 0.4824 - val_accuracy: 0.7872


In [18]:
# Get results
y_pred_1 = tf.squeeze(tf.round(model_1.predict(X_test)))
calculate_results(y_test,y_pred_1)

{'accuracy': 78.7215411558669,
 'f1': 0.7836218881747316,
 'precision': 0.7915324377310113,
 'recall': 0.787215411558669}

# Visualize learned embeddings

In [19]:
# Get the vocabulary from the text vectorization layer
words = text_vectorizer.get_vocabulary()

In [20]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

The weight matrix of the embedding layer contains the numerical representations of each token in our training data, which have been learned for 5 epochs.

In [21]:
embedding_weights = model_1.get_layer('embedding').get_weights()[0]

In [22]:
# Create embedding files for visualization
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embedding_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model 2 : LSTM

In [23]:
# Create a LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64,return_sequences=True)(x) # stacking
x = layers.LSTM(64)(x)
#x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs,name='model_2_LSTM')

In [24]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                      

In [25]:
model_2.compile(loss='binary_crossentropy',optimizer='Adam',
                metrics=['accuracy'])

In [26]:
# Fit model
model_2_history = model_2.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_test,y_test),
                              callbacks=create_tensorboard_callback(SAVE_DIR,
                                                                    'model_2_LSTM'))

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220516-163523
Epoch 1/5
203/203 [==============================] - 10s 20ms/step - loss: 0.2182 - accuracy: 0.9227 - val_loss: 0.5504 - val_accuracy: 0.7767
Epoch 2/5
203/203 [==============================] - 3s 14ms/step - loss: 0.1536 - accuracy: 0.9437 - val_loss: 0.6286 - val_accuracy: 0.7741
Epoch 3/5
203/203 [==============================] - 3s 14ms/step - loss: 0.1258 - accuracy: 0.9543 - val_loss: 0.7104 - val_accuracy: 0.7706
Epoch 4/5
203/203 [==============================] - 3s 15ms/step - loss: 0.1034 - accuracy: 0.9598 - val_loss: 0.7768 - val_accuracy: 0.7627
Epoch 5/5
203/203 [==============================] - 3s 15ms/step - loss: 0.0769 - accuracy: 0.9683 - val_loss: 1.1167 - val_accuracy: 0.7741


In [27]:
model_2_preds = tf.squeeze(tf.round(model_2.predict(X_test)))

calculate_results(y_test,model_2_preds)

{'accuracy': 77.40805604203153,
 'f1': 0.7729642429408159,
 'precision': 0.7735937234911502,
 'recall': 0.7740805604203153}

# Model 3: GRU

In [28]:
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64,return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs,name='model_3_GRU')

In [29]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                       

In [30]:
model_3.compile(loss='binary_crossentropy',optimizer='Adam',
                metrics=['accuracy'])

In [31]:
model_3_history = model_3.fit(X_train,y_train,
            epochs=5,
            validation_data=(X_test,y_test),
            callbacks=create_tensorboard_callback(SAVE_DIR,'model_3_GRU'))

Saving TensorBoard log files to: model_logs/model_3_GRU/20220516-163549
Epoch 1/5
203/203 [==============================] - 11s 27ms/step - loss: 0.1532 - accuracy: 0.9458 - val_loss: 0.7899 - val_accuracy: 0.7767
Epoch 2/5
203/203 [==============================] - 3s 15ms/step - loss: 0.0822 - accuracy: 0.9700 - val_loss: 0.7620 - val_accuracy: 0.7644
Epoch 3/5
203/203 [==============================] - 3s 15ms/step - loss: 0.0659 - accuracy: 0.9747 - val_loss: 1.2688 - val_accuracy: 0.7706
Epoch 4/5
203/203 [==============================] - 3s 14ms/step - loss: 0.0518 - accuracy: 0.9776 - val_loss: 1.5890 - val_accuracy: 0.7426
Epoch 5/5
203/203 [==============================] - 3s 14ms/step - loss: 0.0469 - accuracy: 0.9767 - val_loss: 1.2632 - val_accuracy: 0.7671


In [32]:
model_3_preds = tf.squeeze(tf.round(model_3.predict(X_test)))

calculate_results(y_test,model_3_preds)

{'accuracy': 76.70753064798599,
 'f1': 0.7649114938571001,
 'precision': 0.7674054284930486,
 'recall': 0.7670753064798599}

# Model 4: Bidirectional

In [33]:
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')

In [34]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [35]:
model_4.compile(loss='binary_crossentropy',optimizer='Adam',
                metrics=['accuracy'])

In [36]:
model_4_history = model_4.fit(X_train,y_train,
                              epochs=5,
                              validation_data=(X_test,y_test),
                              callbacks=create_tensorboard_callback(SAVE_DIR,'model_4_bidirectional'))

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220516-163616
Epoch 1/5
203/203 [==============================] - 16s 45ms/step - loss: 0.0976 - accuracy: 0.9700 - val_loss: 0.9188 - val_accuracy: 0.7758
Epoch 2/5
203/203 [==============================] - 7s 33ms/step - loss: 0.0516 - accuracy: 0.9765 - val_loss: 1.3627 - val_accuracy: 0.7478
Epoch 3/5
203/203 [==============================] - 6s 28ms/step - loss: 0.0445 - accuracy: 0.9768 - val_loss: 1.2825 - val_accuracy: 0.7750
Epoch 4/5
203/203 [==============================] - 5s 24ms/step - loss: 0.0398 - accuracy: 0.9791 - val_loss: 1.4211 - val_accuracy: 0.7644
Epoch 5/5
203/203 [==============================] - 5s 24ms/step - loss: 0.0370 - accuracy: 0.9808 - val_loss: 1.6781 - val_accuracy: 0.7750


In [37]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(X_test)))

calculate_results(y_test,model_4_preds)

{'accuracy': 77.49562171628722,
 'f1': 0.7732582150276275,
 'precision': 0.7749850420227321,
 'recall': 0.7749562171628721}

# Convolutional Neural Networks for text

## Model 5

In [40]:
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,kernel_size=5,
                  activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs,name='model_5_Conv1D')

# Compile
model_5.compile(loss='binary_crossentropy',
                optimizer='Adam',metrics=['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [41]:
model_5_history = model_5.fit(X_train,y_train,
                              epochs=5,
                              validation_data=(X_test,y_test),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220516-164912
Epoch 1/5
203/203 [==============================] - 11s 13ms/step - loss: 0.1233 - accuracy: 0.9595 - val_loss: 0.8913 - val_accuracy: 0.7741
Epoch 2/5
203/203 [==============================] - 3s 12ms/step - loss: 0.0712 - accuracy: 0.9734 - val_loss: 1.0357 - val_accuracy: 0.7680
Epoch 3/5
203/203 [==============================] - 3s 14ms/step - loss: 0.0575 - accuracy: 0.9773 - val_loss: 1.0876 - val_accuracy: 0.7706
Epoch 4/5
203/203 [==============================] - 2s 12ms/step - loss: 0.0514 - accuracy: 0.9784 - val_loss: 1.2022 - val_accuracy: 0.7767
Epoch 5/5
203/203 [==============================] - 2s 12ms/step - loss: 0.0482 - accuracy: 0.9799 - val_loss: 1.1793 - val_accuracy: 0.7627


In [42]:
model_5_preds = tf.squeeze(tf.round(model_5.predict(X_test)))

calculate_results(y_test,model_5_preds)

{'accuracy': 76.26970227670753,
 'f1': 0.7615624856623218,
 'precision': 0.762056504701221,
 'recall': 0.7626970227670753}

# Model 6: Pretrained (TF Hub)

In [44]:
import tensorflow_hub as hub

In [45]:
# Create a Keras Layer using the USE pretrained layer 
sentence_encoder = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                  input_shape=[],dtype='string',trainable=False,
                                  name='USE')

In [53]:
# Create model using Sequential API
model_6 = tf.keras.Sequential([sentence_encoder,
                               layers.Dense(64,activation='relu'),
                               layers.Dense(1,activation='sigmoid')],
                              name='model_6_USE')
model_6.compile(loss='binary_crossentropy',
                optimizer='Adam',metrics=['accuracy'])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [54]:
model_6_history = model_6.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'USE')])

Saving TensorBoard log files to: model_logs/USE/20220516-171603
Epoch 1/5
203/203 [==============================] - 5s 21ms/step - loss: 0.5089 - accuracy: 0.7855 - val_loss: 0.4595 - val_accuracy: 0.7925
Epoch 2/5
203/203 [==============================] - 4s 19ms/step - loss: 0.4116 - accuracy: 0.8170 - val_loss: 0.4556 - val_accuracy: 0.7960
Epoch 3/5
203/203 [==============================] - 4s 18ms/step - loss: 0.3972 - accuracy: 0.8209 - val_loss: 0.4518 - val_accuracy: 0.7977
Epoch 4/5
203/203 [==============================] - 4s 18ms/step - loss: 0.3894 - accuracy: 0.8266 - val_loss: 0.4520 - val_accuracy: 0.8082
Epoch 5/5
203/203 [==============================] - 4s 19ms/step - loss: 0.3847 - accuracy: 0.8283 - val_loss: 0.4496 - val_accuracy: 0.8047


In [55]:
model_6_preds = tf.squeeze(tf.round(model_6.predict(X_test)))

calculate_results(y_test,model_6_preds)

{'accuracy': 80.47285464098074,
 'f1': 0.8036023364361472,
 'precision': 0.8048113712814872,
 'recall': 0.8047285464098074}

## The same but with 10% of data

In [62]:
model_7 = tf.keras.models.clone_model(model_6)

In [58]:
# Esto está mal, algunas de las muestras del entrenamiento quedan adentro de la validación.
train_01 = train_df_shuffled[['text','target']].sample(frac=0.1,random_state=42)

X_train_01 = train_01['text'].to_list()
y_train_01 = train_01['target'].to_list()

In [63]:
model_7.compile(loss='binary_crossentropy',optimizer='Adam',
                metrics=['accuracy'])

model_7.fit(X_train_01,y_train_01,
            epochs=5,validation_data=(X_test,y_test),
            callbacks=[create_tensorboard_callback(SAVE_DIR,'USE_10p')])

Saving TensorBoard log files to: model_logs/USE_10p/20220516-172537
Epoch 1/5
24/24 [==============================] - 8s 119ms/step - loss: 0.6725 - accuracy: 0.6518 - val_loss: 0.6371 - val_accuracy: 0.7881
Epoch 2/5
24/24 [==============================] - 2s 83ms/step - loss: 0.5983 - accuracy: 0.7871 - val_loss: 0.5650 - val_accuracy: 0.7872
Epoch 3/5
24/24 [==============================] - 2s 90ms/step - loss: 0.5223 - accuracy: 0.7937 - val_loss: 0.4997 - val_accuracy: 0.7872
Epoch 4/5
24/24 [==============================] - 2s 87ms/step - loss: 0.4639 - accuracy: 0.7989 - val_loss: 0.4611 - val_accuracy: 0.7960
Epoch 5/5
24/24 [==============================] - 2s 86ms/step - loss: 0.4269 - accuracy: 0.8134 - val_loss: 0.4379 - val_accuracy: 0.7986
